In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelBinarizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import train_test_split
from sklearn import neighbors
import datetime
import gc

c:\users\scarlet\appdata\local\programs\python\python35\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
def add_date_features(df):
    df["date"] = pd.to_datetime(df["time_stamp"])
    df["date_year"] = df["date"].dt.year
    df["date_month"] = df["date"].dt.month
    df["date_day"] = df["date"].dt.day
    df["date_hour"] = df["date"].dt.hour
    df["date_min"] = df["date"].dt.minute
    return df

In [7]:
shop_info = pd.read_csv('train_ccf_first_round_shop_info.csv')
user_shop_info = pd.read_csv('train_ccf_first_round_user_shop_behavior.csv')
testA_B = pd.read_csv('AB_test_evaluation_public.csv') 

In [8]:
add_date_features(user_shop_info)
lb = LabelBinarizer()
user_shop_info['wifi_infos'] = user_shop_info['wifi_infos'].str.split(';', expand=True)[0]
user_shop_info['wifi_id'] = user_shop_info['wifi_infos'].str.split('|', expand=True)[0]
user_shop_info['wifi_signal'] = user_shop_info['wifi_infos'].str.split('|', expand=True)[1]
user_shop_info['wifi_flag'] = user_shop_info['wifi_infos'].str.split('|', expand=True)[2]
        
user_shop_info['wifi_flag'] = lb.fit_transform(user_shop_info['wifi_flag'])
user_shop_info = user_shop_info.drop(['wifi_infos'], axis=1)

user_shop_info['wifi_signal'] = user_shop_info['wifi_signal'].astype(np.int32)
user_shop_info['date_day'] = user_shop_info['date_day'].astype(np.int32)
user_shop_info['date_hour'] = user_shop_info['date_hour'].astype(np.int32)
user_shop_info['date_min'] = user_shop_info['date_min'].astype(np.int32)
user_shop_info['longitude'] = user_shop_info['longitude'].astype(np.float32)
user_shop_info['latitude'] = user_shop_info['latitude'].astype(np.float32)
user_shop_info['wifi_id'] = user_shop_info['wifi_id'].str.split('b_', expand=True)[1].astype(np.int32)

user_shop_info = user_shop_info.drop(['date', 'time_stamp', 'date_year', 'date_month', 'user_id'], axis=1)
user_shop_info

,shop_id,longitude,latitude,date_day,date_hour,date_min,wifi_id,wifi_signal,wifi_flag
0,s_2871718,122.308289,32.088039,6,21,20,6396480,-67,0
1,s_2871718,122.308159,32.087971,6,21,20,6396480,-67,0
2,s_181637,117.365257,40.638214,2,13,10,8006367,-78,0
3,s_609470,121.134453,31.197416,13,12,30,26250579,-73,0
4,s_3816766,122.255867,31.351320,25,19,50,39004150,-66,0
5,s_3686420,122.595276,31.581963,28,13,10,13784698,-75,0
6,s_89874,104.815315,30.962847,11,14,20,975810,-33,1
7,s_939447,117.913712,39.589680,6,14,50,15263246,-67,0
8,s_939447,117.913765,39.589699,2,17,30,37749132,-77,0
9,s_3698335,117.914185,39.589539,2,18,30,39846139,-73,0


In [9]:
add_date_features(testA_B)
testA_B = testA_B.drop(['date', 'time_stamp', 'date_year', 'date_month', 'user_id'], axis=1)
testA_B['wifi_infos'] = testA_B['wifi_infos'].str.split(';', expand=True)[0]
testA_B['wifi_id'] = testA_B['wifi_infos'].str.split('|', expand=True)[0]
testA_B['wifi_signal'] = testA_B['wifi_infos'].str.split('|', expand=True)[1]
testA_B['wifi_flag'] = testA_B['wifi_infos'].str.split('|', expand=True)[2]

lb = LabelBinarizer()
testA_B['wifi_flag'] = lb.fit_transform(testA_B['wifi_flag'])
testA_B = testA_B.drop(['wifi_infos'], axis=1)
testA_B['wifi_signal'] = testA_B['wifi_signal'].astype(np.int32)
testA_B['date_day'] = testA_B['date_day'].astype(np.int32)
testA_B['date_hour'] = testA_B['date_hour'].astype(np.int32)
testA_B['date_min'] = testA_B['date_min'].astype(np.int32)
testA_B['longitude'] = testA_B['longitude'].astype(np.float32)
testA_B['latitude'] = testA_B['latitude'].astype(np.float32)
testA_B['wifi_id'] = testA_B['wifi_id'].str.split('b_', expand=True)[1].astype(np.int32)
testA_B

,row_id,mall_id,longitude,latitude,date_day,date_hour,date_min,wifi_id,wifi_signal,wifi_flag
0,118742,m_3916,122.141014,39.818848,5,13,0,34366982,-82,0
1,118743,m_5085,118.191910,32.855858,6,13,10,36722251,-81,0
2,118744,m_4033,119.192108,32.424667,6,17,40,30026291,-74,0
3,118745,m_4515,120.612198,34.055248,3,12,10,33412374,-77,0
4,118746,m_7168,116.861992,40.326859,2,20,40,19882704,-77,0
5,118747,m_4079,117.365334,40.638302,1,13,20,33503892,-50,0
6,118748,m_2333,120.494446,36.455074,6,12,20,44437637,-70,0
7,118749,m_1175,120.744362,30.814718,7,21,10,8763036,-91,0
8,118750,m_1409,122.309341,32.086750,3,10,30,2159244,-82,0
9,118751,m_3517,104.417603,31.286158,2,19,0,34749981,-68,0


In [17]:
mall_ids = list( shop_info.ix[:,'mall_id'].unique() )

In [103]:
count = 0
clf = neighbors.KNeighborsClassifier(algorithm='kd_tree', n_neighbors=1)  
for mall_id in mall_ids[:11]:
    
    count = count + 1
    if(count % 10 == 0):
        print(count)
    shop_ids_by_mall_id = shop_info[shop_info.mall_id == mall_id]
    data_by_mall_id = user_shop_info[user_shop_info['shop_id'].isin(shop_ids_by_mall_id['shop_id'])]
    test_data_by_mall_id = testA_B[testA_B['mall_id'] == mall_id]
    test_data_by_mall_id = test_data_by_mall_id.drop(['mall_id', 'row_id'], axis=1)
    X = data_by_mall_id.ix[:, 1:]
    y = data_by_mall_id.ix[:, :1]
    clf.fit(X, y)
    mid_result = pd.DataFrame(pd.Series(clf.predict(test_data_by_mall_id)).values, columns=['shop_id'])
    result = result.append(mid_result, ignore_index=True)
        #print(result.append(pd.Series(mid_result), ignore_index=True))    
result

c:\users\scarlet\appdata\local\programs\python\python35\lib\site-packages\ipykernel\__main__.py:14: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().


10


,shop_id
0,s_240723
1,s_9147
2,s_9147
3,s_3697
4,s_455479
5,s_152386
6,s_486541
7,s_767494
8,s_389825
9,s_149642


In [20]:
 n_classes = len(data_with_mall_id.ix[:,'shop_id'].unique())
_n_samples = data_with_mall_id.shape[0]
n_classes

94

In [74]:
mall = pd.DataFrame(shop_info['mall_id'].unique(), columns=['ID'])

In [145]:
len(mall)

97

In [75]:
mall_1 = shop_info[shop_info['mall_id'] == mall['ID'][0]] 

In [76]:
mall_1

,shop_id,category_id,longitude,latitude,price,mall_id
0,s_26,c_4,122.346736,31.833507,57,m_690
29,s_1587,c_1,122.347116,31.834759,60,m_690
34,s_1756,c_27,122.346919,31.833567,44,m_690
41,s_2258,c_38,122.346777,31.833647,50,m_690
43,s_2265,c_38,122.346113,31.834622,50,m_690
55,s_3221,c_49,122.346859,31.833882,73,m_690
75,s_3697,c_30,122.346763,31.834160,56,m_690
155,s_5947,c_26,122.346802,31.833697,52,m_690
199,s_9147,c_29,122.346026,31.833803,50,m_690
259,s_12639,c_49,122.347065,31.834226,59,m_690


In [77]:
add_date_features(user_shop_info)
user_shop_info

,user_id,shop_id,time_stamp,longitude,latitude,wifi_infos,date,date_year,date_month,date_day,date_hour,date_min
0,u_376,s_2871718,2017-08-06 21:20,122.308291,32.088040,b_6396480|-67|false;b_41124514|-86|false;b_287...,2017-08-06 21:20:00,2017,8,6,21,20
1,u_376,s_2871718,2017-08-06 21:20,122.308162,32.087970,b_6396480|-67|false;b_56328155|-73|false;b_411...,2017-08-06 21:20:00,2017,8,6,21,20
2,u_1041,s_181637,2017-08-02 13:10,117.365255,40.638214,b_8006367|-78|false;b_2485110|-52|false;b_3005...,2017-08-02 13:10:00,2017,8,2,13,10
3,u_1158,s_609470,2017-08-13 12:30,121.134451,31.197416,b_26250579|-73|false;b_26250580|-64|false;b_26...,2017-08-13 12:30:00,2017,8,13,12,30
4,u_1654,s_3816766,2017-08-25 19:50,122.255867,31.351320,b_39004150|-66|false;b_39004148|-58|false;b_21...,2017-08-25 19:50:00,2017,8,25,19,50
5,u_2733,s_3686420,2017-08-28 13:10,122.595277,31.581963,b_13784698|-75|false;b_7390858|-86|false;b_358...,2017-08-28 13:10:00,2017,8,28,13,10
6,u_2848,s_89874,2017-08-11 14:20,104.815313,30.962847,b_975810|-33|true;b_975807|-41|false;b_3506304...,2017-08-11 14:20:00,2017,8,11,14,20
7,u_3063,s_939447,2017-08-06 14:50,117.913713,39.589681,b_15263246|-67|false;b_10120963|-76|false;b_41...,2017-08-06 14:50:00,2017,8,6,14,50
8,u_3063,s_939447,2017-08-02 17:30,117.913765,39.589699,b_37749132|-77|false;b_25781349|-63|false;b_25...,2017-08-02 17:30:00,2017,8,2,17,30
9,u_3063,s_3698335,2017-08-02 18:30,117.914182,39.589539,b_39846139|-73|false;b_25673942|-61|false;b_25...,2017-08-02 18:30:00,2017,8,2,18,30


In [78]:
user_shop_info = user_shop_info.drop(['date', 'time_stamp', 'date_year', 'date_month', 'user_id'], axis=1)
user_shop_info['wifi_infos'] = user_shop_info['wifi_infos'].str.split(';', expand=True)[0]
user_shop_info['wifi_id'] = user_shop_info['wifi_infos'].str.split('|', expand=True)[0]
user_shop_info['wifi_signal'] = user_shop_info['wifi_infos'].str.split('|', expand=True)[1]
user_shop_info['wifi_flag'] = user_shop_info['wifi_infos'].str.split('|', expand=True)[2]

lb = LabelBinarizer()
user_shop_info['wifi_flag'] = lb.fit_transform(user_shop_info['wifi_flag'])
user_shop_info = user_shop_info.drop(['wifi_infos'], axis=1)


user_shop_info

,shop_id,longitude,latitude,date_day,date_hour,date_min,wifi_id,wifi_signal,wifi_flag
0,s_2871718,122.308291,32.088040,6,21,20,b_6396480,-67,0
1,s_2871718,122.308162,32.087970,6,21,20,b_6396480,-67,0
2,s_181637,117.365255,40.638214,2,13,10,b_8006367,-78,0
3,s_609470,121.134451,31.197416,13,12,30,b_26250579,-73,0
4,s_3816766,122.255867,31.351320,25,19,50,b_39004150,-66,0
5,s_3686420,122.595277,31.581963,28,13,10,b_13784698,-75,0
6,s_89874,104.815313,30.962847,11,14,20,b_975810,-33,1
7,s_939447,117.913713,39.589681,6,14,50,b_15263246,-67,0
8,s_939447,117.913765,39.589699,2,17,30,b_37749132,-77,0
9,s_3698335,117.914182,39.589539,2,18,30,b_39846139,-73,0


In [121]:
user_shop_info['wifi_signal'] = user_shop_info['wifi_signal'].astype(np.int32)
user_shop_info['date_day'] = user_shop_info['date_day'].astype(np.int32)
user_shop_info['date_hour'] = user_shop_info['date_hour'].astype(np.int32)
user_shop_info['date_min'] = user_shop_info['date_min'].astype(np.int32)
user_shop_info['longitude'] = user_shop_info['longitude'].astype(np.float32)
user_shop_info['latitude'] = user_shop_info['latitude'].astype(np.float32)
user_shop_info['wifi_id'] = user_shop_info['wifi_id'].str.split('b_', expand=True)[1].astype(np.int32)
user_shop_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1138015 entries, 0 to 1138014
Data columns (total 9 columns):
shop_id        1138015 non-null object
longitude      1138015 non-null float32
latitude       1138015 non-null float32
date_day       1138015 non-null int32
date_hour      1138015 non-null int32
date_min       1138015 non-null int32
wifi_id        1138015 non-null int32
wifi_signal    1138015 non-null int32
wifi_flag      1138015 non-null int32
dtypes: float32(2), int32(6), object(1)
memory usage: 43.4+ MB


In [122]:
x_train_1 = user_shop_info[user_shop_info['shop_id'].isin(mall_1['shop_id'])] 

In [123]:
x_train_1

,shop_id,longitude,latitude,date_day,date_hour,date_min,wifi_id,wifi_signal,wifi_flag
10,s_149642,122.346924,31.834587,15,9,40,33798074,-90,0
43,s_149642,122.346939,31.834576,6,13,50,25145248,-55,0
56,s_767494,122.347305,31.833839,10,12,30,4121837,-53,0
57,s_5947,122.347343,31.833675,10,12,30,15853307,-69,0
112,s_9147,122.346779,31.833582,12,8,10,18988859,-62,1
113,s_9147,122.346550,31.834223,13,9,0,22076003,-79,0
134,s_298312,122.346680,31.834848,11,18,20,7446873,-68,0
145,s_1425466,122.346092,31.834360,10,17,30,13587603,-63,0
194,s_149642,122.346924,31.834623,6,19,20,22076003,-64,0
195,s_2833968,122.347031,31.835049,6,20,30,15412066,-79,0


In [127]:
x_train, x_valid, y_train, y_valid = train_test_split(x_train_1, y_train, test_size = 0.19)  

In [138]:
clf = neighbors.KNeighborsClassifier(algorithm='kd_tree', n_neighbors=5)  
clf.fit(x_train, y_train)

KNeighborsClassifier(algorithm='kd_tree', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')

In [139]:
answer = clf.predict(test)

In [140]:
answer

array(['s_240723', 's_9147', 's_9147', ..., 's_12639', 's_3345001',
       's_1091614'], dtype=object)

In [141]:
answer1 = clf.predict(x_valid)

In [142]:
len(answer1)

5096

In [143]:
len(y_valid)

5096

In [144]:
count = 0
for i in range(len(answer1)):
    if answer1[i] == y_valid[i]:
        count = count + 1
        
print(count/len(answer1))

0.4872448979591837
